In [1]:
# read grid information from yaml file
import yaml

fname = 'input_file_example.yml'

with open(fname, 'r') as f:
    data = yaml.load(f, Loader=yaml.FullLoader)

    min_max_dep = data['domain']['min_max_dep']
    min_max_lat = data['domain']['min_max_lat']
    min_max_lon = data['domain']['min_max_lon']
    n_rtp = data['domain']['n_rtp']

In [2]:
import numpy as np

# function WGS84 to cartesian
def WGS84_to_cartesian(lat, lon, dep, c_lat, c_lon):
    """
    Convert WGS84 coordinates to cartesian coordinates
    """

    # equatorial radius WGS84 major axis
    equRadius = 6371.0
    flattening = 1.0 / 298.257222101
    sqrEccentricity = flattening * (2.0 - flattening)

    # convert to radians
    lat = np.deg2rad(lat)
    lon = np.deg2rad(lon)
    # convert depth to altitude
    alt = -dep

    sinLat = np.sin(lat)
    cosLat = np.cos(lat)
    sinLon = np.sin(lon)
    cosLon = np.cos(lon)

    # Normalize the radius of curvature
    normRadius = equRadius / np.sqrt(1.0 - sqrEccentricity * sinLat * sinLat)

    # convert to cartesian coordinates
    x = (normRadius + alt) * cosLat * cosLon
    y = (normRadius + alt) * cosLat * sinLon
    z = (normRadius * (1.0 - sqrEccentricity) + alt) * sinLat

    return x, y, z

In [6]:
# make coordinate arrays
import numpy as np
# create element connection
n_el = (n_rtp[0] - 1) * (n_rtp[1] - 1) * (n_rtp[2] - 1)
n_nod = n_rtp[0] * n_rtp[1] * n_rtp[2]

dep = np.linspace(min_max_dep[0], min_max_dep[1], n_rtp[0])
lat = np.linspace(min_max_lat[0], min_max_lat[1], n_rtp[1])
lon = np.linspace(min_max_lon[0], min_max_lon[1], n_rtp[2])

dep1d = np.zeros(n_nod)
lat1d = np.zeros(n_nod)
lon1d = np.zeros(n_nod)
dep1d_cart = np.zeros(n_nod)
lat1d_cart = np.zeros(n_nod)
lon1d_cart = np.zeros(n_nod)

dep3d = np.zeros((n_rtp[0], n_rtp[1], n_rtp[2]))
lat3d = np.zeros((n_rtp[0], n_rtp[1], n_rtp[2]))
lon3d = np.zeros((n_rtp[0], n_rtp[1], n_rtp[2]))
dep3d_cart = np.zeros((n_rtp[0], n_rtp[1], n_rtp[2]))
lat3d_cart = np.zeros((n_rtp[0], n_rtp[1], n_rtp[2]))
lon3d_cart = np.zeros((n_rtp[0], n_rtp[1], n_rtp[2]))

lon_center = (min_max_lon[0] + min_max_lon[1]) / 2
lat_center = (min_max_lat[0] + min_max_lat[1]) / 2

offset = 0

for i in range(n_rtp[0]):
    for j in range(n_rtp[1]):
        for k in range(n_rtp[2]):
            dep1d[offset] = dep[i]
            lat1d[offset] = lat[j]
            lon1d[offset] = lon[k]

            # convert to cartesian coordinates
            x, y, z = WGS84_to_cartesian(lat[j], lon[k], dep[i], lat_center, lon_center)

            dep1d_cart[offset] = z
            lat1d_cart[offset] = y
            lon1d_cart[offset] = x

            dep3d[i,j,k] = dep[i]
            lat3d[i,j,k] = lat[j]
            lon3d[i,j,k] = lon[k]

            dep3d_cart[i,j,k] = z
            lat3d_cart[i,j,k] = y
            lon3d_cart[i,j,k] = x

            offset += 1
            
# create element connection
import numpy as np

el_conn = np.zeros((n_el*9), dtype=int)

# vtk element type id
el_type = 9
offset=0

for i in range(n_rtp[0] - 1):
    for j in range(n_rtp[1] - 1):
        for k in range(n_rtp[2] - 1):
            el_conn[offset] = el_type
            el_conn[offset+1] = i * n_rtp[1] * n_rtp[2] + j * n_rtp[2] + k
            el_conn[offset+2] = i * n_rtp[1] * n_rtp[2] + j * n_rtp[2] + k + 1
            el_conn[offset+3] = i * n_rtp[1] * n_rtp[2] + (j + 1) * n_rtp[2] + k + 1
            el_conn[offset+4] = i * n_rtp[1] * n_rtp[2] + (j + 1) * n_rtp[2] + k
            el_conn[offset+5] = (i + 1) * n_rtp[1] * n_rtp[2] + j * n_rtp[2] + k
            el_conn[offset+6] = (i + 1) * n_rtp[1] * n_rtp[2] + j * n_rtp[2] + k + 1
            el_conn[offset+7] = (i + 1) * n_rtp[1] * n_rtp[2] + (j + 1) * n_rtp[2] + k + 1
            el_conn[offset+8] = (i + 1) * n_rtp[1] * n_rtp[2] + (j + 1) * n_rtp[2] + k
            offset += 9

# create 1d 

In [7]:
# output grid file in h5 format 
import h5py

with h5py.File('grid_init.h5', 'w') as f:
    f.create_dataset('lon', data=lon3d)
    f.create_dataset('lat', data=lat3d)
    f.create_dataset('dep', data=dep3d)
    f.create_dataset('x', data=lon3d_cart)
    f.create_dataset('y', data=lat3d_cart)
    f.create_dataset('z', data=dep3d_cart)
    f.create_dataset('el_conn', data=el_conn)


In [8]:
# output xdmf file for paraview visualization
# grid data is stored in fgrid
# lon lat dep in 3d each array
# field values are stored in fmodel
fgrid = 'grid_init.h5'
fmodel='test_model_true.h5'

import xml.etree.ElementTree as ET

# create xdmf file
root = ET.Element('Xdmf', Version='3.0')
domain = ET.SubElement(root, 'Domain', name='Mesh')

# create Tomology with Mixed type
topology = ET.SubElement(domain, 'Topology', NumberOfElements=str(n_el), TopologyType='Mixed')
# create dataitem
dataitem = ET.SubElement(topology, 'DataItem', Dimensions="{} {}".format(n_el, 9), Format='HDF', NumberType='Int', Precision='4')
dataitem.text = fgrid + ':/el_conn'

# create geometry
geometry = ET.SubElement(domain, 'Geometry', GeometryType='X_Y_Z')
# create dataitem
dataitem = ET.SubElement(geometry, 'DataItem', Dimensions="{} {} {}".format(n_rtp[0], n_rtp[1], n_rtp[2]), Format='HDF', NumberType='Float', Precision='8')
dataitem.text = fgrid + ':/x'
dataitem = ET.SubElement(geometry, 'DataItem', Dimensions="{} {} {}".format(n_rtp[0], n_rtp[1], n_rtp[2]), Format='HDF', NumberType='Float', Precision='8')
dataitem.text = fgrid + ':/y'
dataitem = ET.SubElement(geometry, 'DataItem', Dimensions="{} {} {}".format(n_rtp[0], n_rtp[1], n_rtp[2]), Format='HDF', NumberType='Float', Precision='8')
dataitem.text = fgrid + ':/z'

# create Data grid
grid = ET.SubElement(domain, 'Grid', Name='Data', GridType='Uniform')
# create topology reference
topology = ET.SubElement(grid, 'Topology', Reference='/Xdmf/Domain/Topology')
# create geometry reference
geometry = ET.SubElement(grid, 'Geometry', Reference='/Xdmf/Domain/Geometry')

# create attribute
attribute = ET.SubElement(grid, 'Attribute', Name='vel', AttributeType='Scalar', Center='Node')
# create dataitem
dataitem = ET.SubElement(attribute, 'DataItem', Dimensions="{} {} {}".format(n_rtp[0], n_rtp[1], n_rtp[2]), Format='HDF', NumberType='Float', Precision='8')
dataitem.text = fmodel + ':/vel'

# create attribute
attribute = ET.SubElement(grid, 'Attribute', Name='eta', AttributeType='Scalar', Center='Node')
# create dataitem
dataitem = ET.SubElement(attribute, 'DataItem', Dimensions="{} {} {}".format(n_rtp[0], n_rtp[1], n_rtp[2]), Format='HDF', NumberType='Float', Precision='8')
dataitem.text = fmodel + ':/eta'

# create attribute
attribute = ET.SubElement(grid, 'Attribute', Name='xi', AttributeType='Scalar', Center='Node')
# create dataitem
dataitem = ET.SubElement(attribute, 'DataItem', Dimensions="{} {} {}".format(n_rtp[0], n_rtp[1], n_rtp[2]), Format='HDF', NumberType='Float', Precision='8')
dataitem.text = fmodel + ':/xi'

# write xdmf file with pretty print
tree = ET.ElementTree(root)
ET.indent(tree, space="  ")
tree.write('test_model_init.xmf', encoding='utf-8', xml_declaration=True)


## open test_model_init.xmf with paraview